# 动静统一

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_fallback.ipynb)&emsp;
[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_fallback.py)&emsp;
[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/advance/pynative_graph/fallback.ipynb)

!!zhaoyu, 修改下面这段话：动静统一：MindSpore可以通过set_context来实现动静态图的一键式切换，但是不可否认当前确实存在不少动态图无法转到静态图的语法，MindSpore正在做JIT fallback的工作，预计1.6版本先实现编译推导期的fallback。

## JIT Fallback

JIT Fallback是为了实现动静统一提出的功能特性。通过JIT Fallback等特性，静态图可以支持尽量多的动态图语法，使得静态图提供接近动态图的语法使用体验。

JIT Fallback是从静态图的角度出发考虑静态图和动态图的统一。MindSpore默认使用静态图模式，用户编写程序时需要遵循MindSpore[静态图语法支持](https://www.mindspore.cn/docs/note/zh-CN/master/static_graph_syntax_support.html)，语法使用存在约束限制。在动态图模式下，Python脚本代码会根据Python语法进行执行，用户可以使用任意Python语法。可以看出，静态图和动态图的语法约束限制是不同的。JIT Fallback特性可以使得静态图支持尽量多的动态图语法，用户能够灵活地进行静态图和动态图的切换。

当前JIT Fallback支持静态图模式的部分常量场景，包括在construct/ms_function中调用第三方库、创建及使用Tensor、调用Python的print打印等。更多JIT Fallback的说明和使用，请参考[JIT Fallback文档](https://www.mindspore.cn/tutorials/zh-CN/master/compiler_optimization/jit_fallback.html)。

!!!zhaoyu,好好理解下面的内容，并融入到上面的知识里面：

===========

JIT fallback是用静态图的角度出发来考虑动静图的统一，希望静态图能够尽量多的支持动态图的语法，其思路借鉴了传统JIT编译的fallback的思路，传统的JIT编译经常会通过profiling信息，对函数进行多态选择、value推导、分支调度等优化，同时设置guard条件，一旦guard条件发现情况有变，可以去JIT优化，回到原来未优化的函数进行解释执行。

对于AI框架的JIT fallback没有那么复杂，在静态图编译的时候（一般JIT fallback是基于ast based的静态图），如果发现是编译器不支持的Python语法，可以把相关语句保留下来，生成解释节点，然后在后面的处理中，fallback到python去执行相关的语句，从而实现相关语法的支持，这里有几个难点：

1）不支持的Python语法的识别

2）解释节点的推导和执行，解释节点有两个运行时机：首先是编译期的推导，一般而言，解释节点尽量在这个实际执行；其次是运行期的执行。

===========

代码用例如下，其中，MindSpore静态图模式不支持在construct中调用NumPy第三方库和创建Tensor对象，因此用例中的`x = np.array([1, 2, 3])`和`y = Tensor(x)`将会通过JIT Fallback特性使用Python解释器进行解释执行，从而实现对这些语法的支持。

In [ ]:
import numpy as np
import mindspore.nn as nn
from mindspore import context, Tensor

context.set_context(mode=context.GRAPH_MODE)

class Net(nn.Cell):
    def construct(self):
        x = np.array([1, 2, 3])
        y = Tensor(x)
        return y

net = Net()
print(net())

## 注意事项

在使用ms_function来修饰函数，加速执行效率时，请注意以下几点：

1. ms_function修饰的函数须在静态图编译支持的语法范围内，包括但不限于数据类型等。

2. ms_function修饰的函数所支持的控制流语法，与静态图保持一致。其中，仅对固定循环次数或者分支条件的控制流结构具有加速效果。

3. 在PyNative模式下使用ms_function功能时，非ms_function修饰的部分支持断点调试；被ms_function修饰的部分由于是以静态图的方式编译，不支持断点调试。

4. 由于ms_function修饰的函数将按照静态图的方式编译执行，因此ms_function不支持修饰的函数中含有Hook算子，以及不支持修饰自定义Bprop函数等。

5. ms_function修饰的函数会受到静态图函数副作用的影响。函数副作用指：当调用函数时，除了函数返回值之外，还对主调用函数产生的附加影响，例如修改全局变量（函数外的变量），修改函数的参数等。

场景1：

In [8]:
import numpy as np
from mindspore import context, Tensor, ms_function

# pylint: disable=W0612

value = 5

@ms_function
def func(x, y):
    out = x + y
    value = 1
    return out

context.set_context(mode=context.PYNATIVE_MODE)
x = Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
y = Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
func(x, y)
print(value)

5


该场景下，`value`是全局变量且在`func`函数中被修改。此时，如果用`ms_function`修饰`func`函数，全局变量`value`的值将不会被修改。原因是：**静态图编译时，会优化掉与返回值无关的语句**。

场景2：

In [9]:
import numpy as np
import mindspore.nn as nn
from mindspore import context, Tensor, ms_function

class Func(nn.Cell):
    def __init__(self):
        super(Func, self).__init__()
        self.value = 5

    @ms_function
    def construct(self, x):
        out = self.value + x
        return out

context.set_context(mode=context.PYNATIVE_MODE)
x = Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
func = Func()
print("out1:", func(x))
func.value = 1
print("out2:", func(x))

out1: [6. 7. 8.]
out2: [6. 7. 8.]


从上面的打印可以看出，在修改了Func类的成员变量value的值为1之后，对成员函数construct的操作并无影响。这是因为在此场景下，用ms_function修饰了`Func`对象的`construct`成员函数，执行`Func`时将会以静态图的方式编译执行。由于静态图会缓存编译结果，第二次调用`Func`时，对`value`的修改不会生效。
